In [53]:
import os
import pandas as pd
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List, Dict, Any
from dotenv import load_dotenv
from langchain_community.document_loaders import CSVLoader
from langchain.prompts import PromptTemplate


In [54]:
from dotenv import load_dotenv
import os

# Option 1: Specify the absolute path to the file
load_dotenv('C:\\Users\\TERENTI\\Desktop\\UNI\\LLM\\api_key.env')

# Now you can access the API key
api_key = os.getenv('API_KEY')


In [55]:
#initializing model 
model_name = 'gemini-2.0-flash'


llm_model = ChatGoogleGenerativeAI(
    model=model_name,
    google_api_key=api_key,
    temperature=0.0,
    # Gemini can sometimes benefit from converting system messages
    # if you encounter issues with prompts that rely heavily on system roles.
    convert_system_message_to_human=True
)

print(f"Initialized Gemini Model: {model_name}")


Initialized Gemini Model: gemini-2.0-flash


In [56]:
def data_intro(df):
    # Create profile dictionary
    profile = {
        "shape": df.shape,
        "columns": list(df.columns),
        "dtypes": df.dtypes.to_dict(),
        "missing_values": {col: {"count": int(df[col].isnull().sum()), 
                                "percentage": float(df[col].isnull().mean() * 100)} 
                        for col in df.columns},
        "sample_rows": df.head(3).to_dict(orient='records'),
        "numeric_stats": {}
    }
    
    # Print each section with formatting
    print("\n" + "="*50)
    print("DATA INTRODUCTION")
    print("="*50)
    
    print("\n📊 SHAPE:")
    print(f"    Rows: {profile['shape'][0]}")
    print(f"    Columns: {profile['shape'][1]}")
    
    print("\n📋 COLUMNS:")
    for i, col in enumerate(profile['columns'], 1):
        print(f"    {i}. {col}")
    
    print("\n📝 DATA TYPES:")
    for col, dtype in profile['dtypes'].items():
        print(f"    {col}: {dtype}")
    
    print("\n❓ MISSING VALUES:")
    for col, missing in profile['missing_values'].items():
        print(f"    {col}: {missing['count']} values ({missing['percentage']:.2f}%)")
    
    print("\n📄 SAMPLE ROWS:")
    for i, row in enumerate(profile['sample_rows'], 1):
        print(f"    Row {i}:")
        for col, val in row.items():
            print(f"        {col}: {val}")
        print()
    
    return profile

df = pd.read_csv('reported.csv')
data_intro(df)


DATA INTRODUCTION

📊 SHAPE:
    Rows: 66
    Columns: 21

📋 COLUMNS:
    1. Year
    2. crimes.total
    3. crimes.penal.code
    4. crimes.person
    5. murder
    6. assault
    7. sexual.offenses
    8. rape
    9. stealing.general
    10. burglary
    11. house.theft
    12. vehicle.theft
    13. out.of.vehicle.theft
    14. shop.theft
    15. robbery
    16. fraud
    17. criminal.damage
    18. other.penal.crimes
    19. narcotics
    20. drunk.driving
    21. population

📝 DATA TYPES:
    Year: int64
    crimes.total: int64
    crimes.penal.code: int64
    crimes.person: int64
    murder: int64
    assault: int64
    sexual.offenses: int64
    rape: int64
    stealing.general: int64
    burglary: int64
    house.theft: float64
    vehicle.theft: float64
    out.of.vehicle.theft: float64
    shop.theft: float64
    robbery: int64
    fraud: int64
    criminal.damage: int64
    other.penal.crimes: int64
    narcotics: float64
    drunk.driving: int64
    population: int64

❓ MISS

{'shape': (66, 21),
 'columns': ['Year',
  'crimes.total',
  'crimes.penal.code',
  'crimes.person',
  'murder',
  'assault',
  'sexual.offenses',
  'rape',
  'stealing.general',
  'burglary',
  'house.theft',
  'vehicle.theft',
  'out.of.vehicle.theft',
  'shop.theft',
  'robbery',
  'fraud',
  'criminal.damage',
  'other.penal.crimes',
  'narcotics',
  'drunk.driving',
  'population'],
 'dtypes': {'Year': dtype('int64'),
  'crimes.total': dtype('int64'),
  'crimes.penal.code': dtype('int64'),
  'crimes.person': dtype('int64'),
  'murder': dtype('int64'),
  'assault': dtype('int64'),
  'sexual.offenses': dtype('int64'),
  'rape': dtype('int64'),
  'stealing.general': dtype('int64'),
  'burglary': dtype('int64'),
  'house.theft': dtype('float64'),
  'vehicle.theft': dtype('float64'),
  'out.of.vehicle.theft': dtype('float64'),
  'shop.theft': dtype('float64'),
  'robbery': dtype('int64'),
  'fraud': dtype('int64'),
  'criminal.damage': dtype('int64'),
  'other.penal.crimes': dtype('int

In [57]:
loader = CSVLoader(r'C:\Users\TERENTI\Desktop\UNI\LLM\HW\reported.csv')
data=loader.load()



# --- Inspect the First Document ---
first_doc = data[0]
row_content = first_doc.page_content
row_metadata = first_doc.metadata

print("--- Original Document (First Row) ---")
print(first_doc)
print("\n--- Page Content Only ---")
print(row_content)


--- Original Document (First Row) ---
page_content='Year: 1950
crimes.total: 2784
crimes.penal.code: 2306
crimes.person: 120
murder: 1
assault: 105
sexual.offenses: 40
rape: 5
stealing.general: 1578
burglary: 295
house.theft: NA
vehicle.theft: NA
out.of.vehicle.theft: NA
shop.theft: NA
robbery: 3
fraud: 209
criminal.damage: 72
other.penal.crimes: 477
narcotics: 0
drunk.driving: 49
population: 7014000' metadata={'source': 'C:\\Users\\TERENTI\\Desktop\\UNI\\LLM\\HW\\reported.csv', 'row': 0}

--- Page Content Only ---
Year: 1950
crimes.total: 2784
crimes.penal.code: 2306
crimes.person: 120
murder: 1
assault: 105
sexual.offenses: 40
rape: 5
stealing.general: 1578
burglary: 295
house.theft: NA
vehicle.theft: NA
out.of.vehicle.theft: NA
shop.theft: NA
robbery: 3
fraud: 209
criminal.damage: 72
other.penal.crimes: 477
narcotics: 0
drunk.driving: 49
population: 7014000


In [69]:
#SHOWCASE THAT DOESNT WORKS



from langchain.chains import LLMChain, SimpleSequentialChain

topic_prompt = PromptTemplate(
    input_variable=['inference'],
    template ='go through the given task step by step, take the most significant information and at every step ask critically evaluate you result, and only after that give the answe on the given {inference}, do not count year and total crime rate as the inference, its more like result of this year like it is in the logistic linear regression'
)
chain_inference = LLMChain(llm=llm_model, prompt=topic_prompt)
# print(result)


# topic_prompt2=PromptTemplate(
#     input_variables=['correlation'],
#     template='based on the columns tell me which one of the columns has the heighest {correlation}, also tell me what is their correlation coefficent,Do not count and year correlation, it doesnt make any sense to do so'
# )
# chain_correlation = LLMChain(llm=llm_model, prompt=topic_prompt2)


topic_prompt3 = PromptTemplate(
    input_variables=['loss'],
    template='on the prevois correlation columns ignor the {loss}, and provide this data'
)
chain_clean=LLMChain(llm=llm_model, prompt=topic_prompt3)


topic_prompt4=PromptTemplate(
    input_variables=['prediction'],
    template='NOW, tell what would the crime statistics in 2016 as the {prediction}, for the given data as good as you can, final output must be prediciton its a must'
)
chain_predict = LLMChain(llm=llm_model, prompt=topic_prompt4)

main_chain=SimpleSequentialChain(chains=[chain_inference,chain_clean, chain_predict])
output= main_chain.run(data)
print(output)



c:\Users\TERENTI\AppData\Local\Programs\Python\Python313\Lib\site-packages\langchain_google_genai\chat_models.py:367: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
c:\Users\TERENTI\AppData\Local\Programs\Python\Python313\Lib\site-packages\langchain_google_genai\chat_models.py:367: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")
c:\Users\TERENTI\AppData\Local\Programs\Python\Python313\Lib\site-packages\langchain_google_genai\chat_models.py:367: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Okay, I understand.  Given that I *still* cannot access the data, I will create a **simulated prediction for 2016** based on the simulated trends from my previous response.  This is purely hypothetical and based on the assumptions I made earlier.  It's crucial to remember that this is *not* a real prediction and should not be taken as such.  It's an illustration of how I would use the trends to extrapolate.

**Simulated Prediction for 2016 Crime Statistics**

**Assumptions (Based on Previous Simulated Analysis):**

*   **Overall Crime Rate:**  The overall crime rate (excluding `crimes.total`) has been declining or stabilizing in recent years (let's say from 2000 onwards).
*   **Population Growth:**  Population continues to grow at a moderate rate.
*   **Specific Crime Trends:**
    *   **Murder:**  Continuing a slow decline.
    *   **Assault:**  Stabilizing or slightly increasing.
    *   **Sexual Offenses & Rape:**  Continuing to increase, but at a slower rate than previously.
    * 

In [ ]:
#Second sequantial prompt WORKS better
analysis_template = """
You are a data cleaning assistant. Analyze the following data row extracted from a CSV file.
The row content is:
---
{row_data}
---
The row comes from source '{source}' and is row number {row_num}.

Based on typical data quality standards and the likely structure of this data (inferring columns if possible, e.g., ["Date", "City", "Event Type", "Value", "Notes"]), please identify potential issues:

1.  **Missing Values:** Are there any empty fields, or fields containing placeholders like 'NA', 'None', 'null', '?', '--'?
2.  **Formatting Errors:** Check for inconsistent date/time formats (suggest a standard like YYYY-MM-DD), numbers with extra characters, inconsistent capitalization, leading/trailing whitespace.
3.  **Data Type Issues:** Does any field seem to have the wrong type (e.g., text in a numeric column)?
4.  **Consistency/Validity:** Are there potential typos? Do categorical values seem consistent? Are there values that seem logically impossible (e.g., a future date for a past event)?
5.  **Other Issues:** Anything else that looks suspicious?

For each issue found:
- Describe the problem clearly.
- Indicate the specific field or value affected.
- Suggest a concrete cleaning action (e.g., "Remove leading whitespace", "Convert to YYYY-MM-DD", "Replace 'NA' with 0", "Correct typo 'Cty' to 'City'").

If the row appears clean based on these checks, state "No major issues found."

Analysis Report:
"""

analysis_prompt = ChatPromptTemplate.from_template(analysis_template)

# --- Create and Run the Analysis Chain ---
# Use a simple string output parser to get the LLM's text response
analysis_chain = LLMChain(llm=llm_model, prompt=analysis_prompt, output_parser=StrOutputParser())

print("\n--- Requesting Cleaning Analysis from LLM ---")
try:
    # Provide the row content and metadata to the prompt
    analysis_result = analysis_chain.invoke({
        "row_data": row_content,
        "source": row_metadata.get('source', 'N/A'), # Safely get metadata
        "row_num": row_metadata.get('row', 'N/A')
    })

    print("\n--- LLM Analysis Result ---")
    print(analysis_result)

except Exception as e:
    print(f"\nError invoking LLM chain: {e}")


    


--- Requesting Cleaning Analysis from LLM ---


c:\Users\TERENTI\AppData\Local\Programs\Python\Python313\Lib\site-packages\langchain_google_genai\chat_models.py:367: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



--- LLM Analysis Result ---
{'row_data': 'Year: 1950\ncrimes.total: 2784\ncrimes.penal.code: 2306\ncrimes.person: 120\nmurder: 1\nassault: 105\nsexual.offenses: 40\nrape: 5\nstealing.general: 1578\nburglary: 295\nhouse.theft: NA\nvehicle.theft: NA\nout.of.vehicle.theft: NA\nshop.theft: NA\nrobbery: 3\nfraud: 209\ncriminal.damage: 72\nother.penal.crimes: 477\nnarcotics: 0\ndrunk.driving: 49\npopulation: 7014000', 'source': 'C:\\Users\\TERENTI\\Desktop\\UNI\\LLM\\HW\\reported.csv', 'row_num': 0, 'text': 'Okay, let\'s analyze the provided data row for potential data quality issues.\n\n**Analysis Report:**\n\n1.  **Missing Values:**\n    - **Problem:** Several fields have "NA" which likely represents missing data.\n    - **Affected Fields:** `house.theft`, `vehicle.theft`, `out.of.vehicle.theft`, `shop.theft`\n    - **Cleaning Action:**  Replace "NA" with a more appropriate representation of missing data.  Options include:\n        *   Replacing with `NULL` (if the database supports it).\

In [71]:
# AGENT
import numpy as np
from langchain_core.output_parsers import StrOutputParser

from pydantic import BaseModel, Field
from typing import List, Dict, Any, Optional

# 1. First, load the data properly
df = pd.read_csv('reported.csv')

 

# 3. Create a data inference function using LLM
def analyze_crime_data(query, df):
    """Use LLM to analyze crime data and answer inference questions"""
    
    # Create a summary of the data for context
    data_summary = f"""
    This dataset contains crime statistics from {df['Year'].min()} to {df['Year'].max()}.
    It includes {df.shape[1]} columns with various crime categories and population data.
    Total records: {df.shape[0]}
    
    Some key columns:
    - Year: The year of recorded crimes
    - crimes.total: Total number of reported crimes
    - crimes.person: Crimes against persons
    - murder, assault, rape: Specific violent crime categories
    - stealing.general, burglary, robbery: Property crime categories
    - population: Population count for the year
    
    There are some missing values (NA) in certain categories, particularly:
    - house.theft: {df['house.theft'].isna().sum()} missing values
    - vehicle.theft: {df['vehicle.theft'].isna().sum()} missing values
    - shop.theft: {df['shop.theft'].isna().sum()} missing values
    - out.of.vehicle.theft: {df['out.of.vehicle.theft'].isna().sum()} missing values
    """
    
    # Create a prompt for the LLM
    inference_template = """
    You are a data analyst specialized in crime statistics. Based on the provided dataset information, 
    answer the following query with detailed analysis.
    
    Dataset Summary:
    {data_summary}
    
    User Query: {query}
    
    Provide a thorough analysis based on the data description. If you cannot answer with certainty,
    explain what additional data or analysis would be needed.
    """
    
    inference_prompt = ChatPromptTemplate.from_template(inference_template)
    
    # Create analysis chain
    inference_chain = LLMChain(
        llm=llm_model,
        prompt=inference_prompt,
        output_parser=StrOutputParser()
    )
    
    # Run the analysis
    result = inference_chain.invoke({
        "data_summary": data_summary,
        "query": query
    })
    
    return result

# 4. Create a function to replace NA values with column averages
def replace_na_with_average(df):
    """Replace NA values in each column with the column average"""
    
    # Create a copy of the dataframe to avoid modifying the original
    cleaned_df = df.copy()
    
    # Track what changes were made
    changes_report = {}
    
    # Process each column
    for column in cleaned_df.columns:
        # Check if column has NA values and is numeric
        if cleaned_df[column].isna().any() and np.issubdtype(cleaned_df[column].dtype, np.number):
            # Calculate average excluding NA values
            avg_value = cleaned_df[column].mean()
            # Count NA values
            na_count = cleaned_df[column].isna().sum()
            # Replace NA with average
            cleaned_df[column].fillna(avg_value, inplace=True)
            # Record the change
            changes_report[column] = {
                "na_count": int(na_count),
                "replacement_value": float(avg_value),
                "percentage_affected": float(na_count / len(cleaned_df) * 100)
            }
    
    return cleaned_df, changes_report


In [75]:

# 5. AGENT 
class CrimeDataAgent:
    def __init__(self, df):
        self.df = df
        self.cleaned_df = None
        self.changes_report = None
    
    def analyze(self, query):
        """Answer inference questions about the data"""
        return analyze_crime_data(query, self.df)
    
    def description(self):
        return df.describe()
    
    def clean_data(self):
        """Clean data by replacing NA values with column averages"""
        self.cleaned_df, self.changes_report = replace_na_with_average(self.df)
        return self.changes_report
    
    def get_original_data(self):
        """Return the original dataframe"""
        return self.df
    
    def get_cleaned_data(self):
        """Return the cleaned dataframe"""
        if self.cleaned_df is None:
            self.clean_data()
        return self.cleaned_df
    
    def compare_stats(self, columns=None):
        if self.cleaned_df is None:
            self.clean_data()
        
        if columns is None:
            columns = [col for col in self.df.columns if col in self.changes_report]
        
        comparison = {}
        for col in columns:
            if col in self.changes_report:
                comparison[col] = {
                    "original_mean": float(self.df[col].mean()),
                    "original_std": float(self.df[col].std()),
                    "cleaned_mean": float(self.cleaned_df[col].mean()),
                    "cleaned_std": float(self.cleaned_df[col].std()),
                    "na_replaced": self.changes_report[col]["na_count"]
                }
        
        return comparison

# Example usage:
# Initialize the agent
crime_agent = CrimeDataAgent(df)

# Example 1: Ask an inference question
inference_result = crime_agent.analyze("What are the trends in violent crimes over time?")
print("INFERENCE ANALYSIS:")
print(inference_result)

# Example 2: Clean the data by replacing NA values with averages
changes = crime_agent.clean_data()
print("\nCLEANING REPORT:")
for col, details in changes.items():
    print(f"- {col}: Replaced {details['na_count']} NA values with {details['replacement_value']:.2f}")
    
# Example 3: Compare statistics before and after cleaning
stats_comparison = crime_agent.compare_stats()
print("\nSTATISTICS COMPARISON:")
for col, stats in stats_comparison.items():
    print(f"- {col}:")
    print(f"  Before: mean={stats['original_mean']:.2f}, std={stats['original_std']:.2f}")
    print(f"  After:  mean={stats['cleaned_mean']:.2f}, std={stats['cleaned_std']:.2f}")

c:\Users\TERENTI\AppData\Local\Programs\Python\Python313\Lib\site-packages\langchain_google_genai\chat_models.py:367: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


INFERENCE ANALYSIS:
{'data_summary': '\n    This dataset contains crime statistics from 1950 to 2015.\n    It includes 21 columns with various crime categories and population data.\n    Total records: 66\n\n    Some key columns:\n    - Year: The year of recorded crimes\n    - crimes.total: Total number of reported crimes\n    - crimes.person: Crimes against persons\n    - murder, assault, rape: Specific violent crime categories\n    - stealing.general, burglary, robbery: Property crime categories\n    - population: Population count for the year\n\n    There are some missing values (NA) in certain categories, particularly:\n    - house.theft: 15 missing values\n    - vehicle.theft: 7 missing values\n    - shop.theft: 15 missing values\n    - out.of.vehicle.theft: 15 missing values\n    ', 'query': 'What are the trends in violent crimes over time?', 'text': 'Okay, here\'s an analysis of the trends in violent crimes over time, based on the provided dataset information.\n\n**Analysis of Vi

C:\Users\TERENTI\AppData\Local\Temp\ipykernel_16096\2727392121.py:88: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  cleaned_df[column].fillna(avg_value, inplace=True)
